# XORStringsNet
> Automatically defeating this dotnet string cryptor

- toc: true 
- badges: true
- categories: [dotnet,xorstringsnet,AgentTesla]


## Overview

[Dr4k0nia](https://twitter.com/dr4k0nia) published an open source .NET string encryption tool on her github [XorStringsNET](https://github.com/dr4k0nia/XorStringsNET). This has been picked up and abused by malware developers to encrypt strings in their malware, including families like Redline and AgentTesla. 

In response Dr4k0nia released a decryption method and a Yara rule.

> https://twitter.com/dr4k0nia/status/1640049711431204872?s=20


### References

- [XorStringsNET](https://github.com/dr4k0nia/XorStringsNET)
- [Encrypting strings in .NET](https://dr4k0nia.github.io/posts/Encrypting-Strings-In-NET/)
- [Unpacking XorStringsNET](https://gist.github.com/dr4k0nia/e59a9902a06da3c875333a98fe856082)
- [Yara: msil_susp_obf_xorstringsnet ](https://github.com/dr4k0nia/yara-rules/blob/main/dotnet/msil_susp_obf_xorstringsnet.yar)

### Samples
- [d56f2852762f7f9fcb07eaf018e143ab1e4ad46e1f2e943faf13618388ef21a2](https://www.unpac.me/results/3b879733-fc84-4a5c-ac25-f10ef82da71a?hash=d56f2852762f7f9fcb07eaf018e143ab1e4ad46e1f2e943faf13618388ef21a2#/)


## Analysis
There is a bug with the encryption which results in only one byte of the key being used to encrypt the strings. This means we can brute force the strings... or we could take a more measured approach.

### Step 1
First, locate the strings table in memory using the decryption routine, then locating the nested private struct and a field referencing that struct. In the example below the  RVA: 0x0001BB7B for the field is the start of the strings table. 

```c#
// Fields
// Token: 0x04000187 RID: 391 RVA: 0x0001BB7B File Offset: 0x00019D7B
.field private static valuetype A.u/A bL at I_0001bb7b // 19516 (0x4c3c) bytes
```

### Step 2
The strings table starts with a 4-byte global key that is used to decrypt the decryption function argument (the pointer to the encrypted string data). Following this key is a serialized stream of the encrypted stings using the following format.

```
4 bytes - length little endian
4 bytes - key (only lsb is used)
data
```



In [82]:
import struct

def xor_crypt(data, key_byte):
    out = []
    for c in data:
        out.append(c ^ key_byte)
    return bytes(out)

file_data = open('/tmp/nettest.bin', 'rb').read()

offset = 0x00019D7B
# skip the global enc key
offset += 4

ptr = offset

strings = []

while True:
    size = struct.unpack('<I', file_data[ptr:ptr+4])[0]
    if size == 0:
        break
    if size > 0x100:
        break
    ptr += 4
    key = file_data[ptr:ptr+1]
    ptr += 4
    data = file_data[ptr:ptr+size]
    ptr += size
    out = xor_crypt(data[::-1], ord(key))
    if not out.isascii():
        break
    strings.append(out)


    
    

    


In [74]:

from dnfile import dnPE
from dnfile.mdtable import MethodDefRow

import dnfile
from dnfile.enums import MetadataTables

from dncil.cil.body import CilMethodBody
from dncil.cil.error import MethodBodyFormatError
from dncil.clr.token import Token, StringToken, InvalidToken
from dncil.cil.body.reader import CilMethodBodyReaderBase

# key token indexes to dotnet meta tables
DOTNET_META_TABLES_BY_INDEX = {table.value: table.name for table in MetadataTables}



In [87]:
class_size_list = []

for row in pe.net.mdtables.ClassLayout.rows:
    type_row = pe.net.mdtables.TypeDef.rows[row.Parent.row_index - 1]
    if type_row.Flags.tdExplicitLayout:
         class_size_list.append(row.ClassSize)
            
# Assume the encryption table is the largest class
target_class_size = max(class_size_list)

target_rva = None

# Find the table based on the pysical size
for i in range(0,len(pe.net.mdtables.FieldRva.rows) - 1,2):
    curr_row = pe.net.mdtables.FieldRva.rows[i]
    next_row = pe.net.mdtables.FieldRva.rows[i +1]
    if next_row.Rva - curr_row.Rva == target_class_size:
        target_rva = curr_row.Rva
        break
        
if target_rva is None:
    # It must be the last one
    target_rva = pe.net.mdtables.FieldRva.rows[-1].Rva
    
data = pe.get_data(target_rva, target_class_size)


def decrypt_table(data):
    strings = []
    ptr = 0 
    # Jump past global key
    ptr += 4
    while ptr < len(data):
        size = struct.unpack('<I', data[ptr:ptr+4])[0]
        if size == 0:
            break
        if size > 0x100:
            break
        ptr += 4
        key = data[ptr:ptr+1]
        ptr += 4
        str_data = data[ptr:ptr+size]
        ptr += size
        out = xor_crypt(str_data[::-1], ord(key))
        if not out.isascii():
            break
        strings.append(out)
    return strings


decrypt_table(data)

[b'SC',
 b'/log.tmp',
 b'KL',
 b'KL',
 b'<br>[',
 b'yyyy-MM-dd HH:mm:ss',
 b']<br>',
 b'<br>',
 b'PW',
 b'Time: ',
 b'MM/dd/yyyy HH:mm:ss',
 b'<br>User Name: ',
 b'<br>Computer Name: ',
 b'<br>OSFullName: ',
 b'<br>CPU: ',
 b'<br>RAM: ',
 b'<br>',
 b'IP Address: ',
 b'<br>',
 b'<hr>',
 b'New ',
 b' Recovered!\n\nTime: ',
 b'MM/dd/yyyy HH:mm:ss',
 b'\nUser Name: ',
 b'/',
 b'\nOSFullName: ',
 b'\nCPU: ',
 b'\nRAM: ',
 b'\n',
 b'IP Address: ',
 b'\n',
 b'_',
 b'/',
 b'/',
 b'false',
 b'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0',
 b'false',
 b'false',
 b'false',
 b'false',
 b'false',
 b'false',
 b'20',
 b'20',
 b'1',
 b'false',
 b'587',
 b'false',
 b'mail.expertsconsultgh.co',
 b'oppong@expertsconsultgh.co',
 b'Oppong.2012',
 b'wisdombig57@gmail.com',
 b'false',
 b'false',
 b'appdata',
 b'xFzxn',
 b'xFzxn.exe',
 b'xFzxn',
 b'Type',
 b': ',
 b': ',
 b'<br>',
 b'<hr>',
 b'<br>',
 b'<b>[ ',
 b']</b> (',
 b')<br>',
 b'{BACK}',
 b'{ALT+TAB}',
 b'{ALT+F4}',
